# FSRS4Anki v3.0.0 Optimizer

[![open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-spaced-repetition/fsrs4anki/blob/v3.0.0/fsrs4anki_optimizer.ipynb)

↑ Click the above button to open the optimizer on Google Colab.

> If you can't see the button and are located in the Chinese Mainland, please use a proxy or VPN.

Upload your **Anki Deck Package (.apkg)** file or **Anki Collection Package (.colpkg)** file on the `Left sidebar -> Files`, drag and drop your file in the current directory (not the `sample_data` directory). 

No need to include media. Need to include scheduling information. 

> If you use the latest version of Anki, please check the box `Support older Anki versions (slower/larger files)` when you export.

You can export it via `File -> Export...` or `Ctrl + E` in the main window of Anki.

Then replace the `filename` with yours in the next code cell. And set the `timezone` and `next_day_starts_at` which can be found in your preferences of Anki.

After that, just run all (`Runtime -> Run all` or `Ctrl + F9`) and wait for minutes. You can see the optimal parameters in section **3 Result**. Copy them, replace the parameters in `fsrs4anki_scheduler.js`, and paste them into the custom scheduling of your deck options (require Anki version >= 2.1.55).

**NOTE**: The default output is generated from my review logs. If you find the output is the same as mine, maybe your notebook hasn't run there.

In [1]:
# Here are some settings that you need to replace before running this optimizer.

filename = "ALL__Learning.apkg"
# If you upload deck file, replace it with your deck filename. E.g., ALL__Learning.apkg
# If you upload collection file, replace it with your colpgk filename. E.g., collection-2022-09-18@13-21-58.colpkg

# Replace it with your timezone. I'm in China, so I use Asia/Shanghai.
timezone = 'Asia/Shanghai'

# Replace it with your Anki's setting in Prefernces -> Scheduling.
next_day_starts_at = 4

# Replace it if you don't want the optimizer to use the review logs before a specific date.
revlog_start_date = "2006-10-05"


## 1 Build dataset

### 1.1 Extract Anki collection & deck file

In [2]:
import zipfile
# Extract the collection file or deck file to get the .anki21 database.
with zipfile.ZipFile(f'./{filename}', 'r') as zip_ref:
    zip_ref.extractall('./')
    print("Extract successfully!")


Extract successfully!


### 1.2 Create time-series feature

The following code cell will extract the review logs from your Anki collection and preprocess them to a trainset which is saved in `revlog_history.tsv`.

 The time-series features are important in optimizing the model's parameters. For more detail, please see my paper: https://www.maimemo.com/paper/

In [3]:
import sqlite3
import time
import tqdm
import pandas as pd
import os
from datetime import timedelta, datetime
from tqdm import tqdm

if os.path.isfile("collection.anki21b"):
    os.remove("collection.anki21b")
    raise Exception(
        "Please export the file with `support older Anki versions` if you use the latest version of Anki.")
elif os.path.isfile("collection.anki21"):
    con = sqlite3.connect("collection.anki21")
elif os.path.isfile("collection.anki2"):
    con = sqlite3.connect("collection.anki2")
else:
    raise Exception("Collection not exist!")
cur = con.cursor()
res = cur.execute("SELECT * FROM revlog")
revlog = res.fetchall()

df = pd.DataFrame(revlog)
df.columns = ['id', 'cid', 'usn', 'r', 'ivl',
              'last_lvl', 'factor', 'time', 'type']
df = df[(df['cid'] <= time.time() * 1000) &
        (df['id'] <= time.time() * 1000) &
        (df['id'] >= time.mktime(datetime.strptime(revlog_start_date, "%Y-%m-%d").timetuple()) * 1000)].copy()
df['create_date'] = pd.to_datetime(df['cid'] // 1000, unit='s')
df['create_date'] = df['create_date'].dt.tz_localize(
    'UTC').dt.tz_convert(timezone)
df['review_date'] = pd.to_datetime(df['id'] // 1000, unit='s')
df['review_date'] = df['review_date'].dt.tz_localize(
    'UTC').dt.tz_convert(timezone)
df.drop(df[df['review_date'].dt.year < 2006].index, inplace=True)
df.sort_values(by=['cid', 'id'], inplace=True, ignore_index=True)
df.to_csv("revlog.csv", index=False)
print("revlog.csv saved!")
df = df[(df['type'] == 0) | (df['type'] == 1)].copy()
df['real_days'] = df['review_date'] - timedelta(hours=next_day_starts_at)
df['real_days'] = pd.DatetimeIndex(df['real_days'].dt.floor('D')).to_julian_date()
df.drop_duplicates(['cid', 'real_days'], keep='first', inplace=True)
df['delta_t'] = df.real_days.diff()
df.dropna(inplace=True)
df['delta_t'] = df['delta_t'].astype(dtype=int)
df['i'] = 1
df['r_history'] = ""
df['t_history'] = ""
col_idx = {key: i for i, key in enumerate(df.columns)}


# code from https://github.com/L-M-Sherlock/anki_revlog_analysis/blob/main/revlog_analysis.py
def get_feature(x):
    for idx, log in enumerate(x.itertuples()):
        if idx == 0:
            x.iloc[idx, col_idx['delta_t']] = 0
        if idx == x.shape[0] - 1:
            break
        x.iloc[idx + 1, col_idx['i']] = x.iloc[idx, col_idx['i']] + 1
        x.iloc[idx + 1, col_idx['t_history']] = f"{x.iloc[idx, col_idx['t_history']]},{x.iloc[idx, col_idx['delta_t']]}"
        x.iloc[idx + 1, col_idx['r_history']] = f"{x.iloc[idx, col_idx['r_history']]},{x.iloc[idx, col_idx['r']]}"
    return x


tqdm.pandas()
df = df.groupby('cid', as_index=False).progress_apply(get_feature)
df["t_history"] = df["t_history"].map(lambda x: x[1:] if len(x) > 1 else x)
df["r_history"] = df["r_history"].map(lambda x: x[1:] if len(x) > 1 else x)
df.to_csv('revlog_history.tsv', sep="\t", index=False)
print("Trainset saved!")


revlog.csv saved!


100%|██████████| 5166/5166 [00:16<00:00, 322.61it/s]


Trainset saved!


## 2 Optimize parameter

### 2.1 Define the model

FSRS is a time-series model for predicting memory states.

In [4]:
import math
import sys
import torch
import numpy as np
from torch import nn
from sklearn.utils import shuffle

w = [1, 1, 5, -1, -1, 0.1, 1.5, -0.2, 0.8, 2, -0.2, 0.2, 1]


class FSRS(nn.Module):
    def __init__(self):
        super(FSRS, self).__init__()
        self.w = nn.Parameter(torch.FloatTensor(w))
        self.zero = torch.FloatTensor([0.0])

    def forward(self, x, s, d):
        '''
        :param x: [review interval, review response]
        :param s: stability
        :param d: difficulty
        :return:
        '''
        if torch.equal(s, self.zero):
            # first learn, init memory states
            new_s = self.w[0] + self.w[1] * (x[1] - 1)
            new_d = self.w[2] + self.w[3] * (x[1] - 3)
            new_d = new_d.clamp(1, 10)
        else:
            r = torch.exp(np.log(0.9) * x[0] / s)
            new_d = d + self.w[4] * (x[1] - 3)
            new_d = self.mean_reversion(self.w[2], new_d)
            new_d = new_d.clamp(1, 10)
            # recall
            if x[1] > 1:
                new_s = s * (1 + torch.exp(self.w[6]) *
                             (11 - new_d) *
                             torch.pow(s, self.w[7]) *
                             (torch.exp((1 - r) * self.w[8]) - 1))
            # forget
            else:
                new_s = self.w[9] * torch.pow(new_d, self.w[10]) * torch.pow(
                    s, self.w[11]) * torch.exp((1 - r) * self.w[12])
        return new_s, new_d

    def loss(self, s, t, r):
        return - (r * np.log(0.9) * t / s + (1 - r) * torch.log(1 - torch.exp(np.log(0.9) * t / s)))

    def mean_reversion(self, init, current):
        return self.w[5] * init + (1-self.w[5]) * current


class WeightClipper(object):
    def __init__(self, frequency=1):
        self.frequency = frequency

    def __call__(self, module):
        if hasattr(module, 'w'):
            w = module.w.data
            w[0] = w[0].clamp(0.1, 10)  # initStability
            w[1] = w[1].clamp(0.1, 5)  # initStabilityRatingFactor
            w[2] = w[2].clamp(1, 10)  # initDifficulty
            w[3] = w[3].clamp(-5, -0.5)  # initDifficultyRatingFactor
            w[4] = w[4].clamp(-5, -0.5)  # updateDifficultyRatingFactor
            w[5] = w[5].clamp(0, 0.5)  # difficultyMeanReversionFactor
            w[6] = w[6].clamp(0, 5)  # recallFactor
            w[7] = w[7].clamp(-0.2, -0.01)  # recallStabilityDecay
            w[8] = w[8].clamp(0.01, 2)  # recallRetrievabilityFactor
            w[9] = w[9].clamp(0.5, 5)  # forgetFactor
            w[10] = w[10].clamp(-2, -0.01)  # forgetDifficultyDecay
            w[11] = w[11].clamp(0.01, 1)  # forgetStabilityDecay
            w[12] = w[12].clamp(0.01, 2)  # forgetRetrievabilityFactor
            module.w.data = w

def lineToTensor(line):
    ivl = line[0].split(',')
    response = line[1].split(',')
    tensor = torch.zeros(len(response), 2)
    for li, response in enumerate(response):
        tensor[li][0] = int(ivl[li])
        tensor[li][1] = int(response)
    return tensor


### 2.2 Train the model

The `revlog_history.tsv` generated before will be used for training the FSRS model.

In [5]:
model = FSRS()
clipper = WeightClipper()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

dataset = pd.read_csv("./revlog_history.tsv", sep='\t', index_col=None)
dataset = dataset[(dataset['i'] > 1) & (dataset['delta_t'] > 0) & (dataset['t_history'].str.count(',0') == 0)]
dataset['tensor'] = dataset.progress_apply(lambda x: lineToTensor(
    list(zip([x['t_history']], [x['r_history']]))[0]), axis=1)
print("Tensorized!")

pre_train_set = dataset[dataset['i'] == 2]
# pretrain
epoch_len = len(pre_train_set)
n_epoch = 1
pbar = tqdm(desc="pre—train", colour="red", total=epoch_len*n_epoch)

for k in range(n_epoch):
    for i, (_, row) in enumerate(shuffle(pre_train_set, random_state=2022 + k).iterrows()):
        model.train()
        optimizer.zero_grad()
        output_t = [(model.zero, model.zero)]
        for input_t in row['tensor']:
            output_t.append(model(input_t, *output_t[-1]))
        loss = model.loss(output_t[-1][0], row['delta_t'],
                            {1: 0, 2: 1, 3: 1, 4: 1}[row['r']])
        if np.isnan(loss.data.item()):
            # Exception Case
            print(row, output_t)
            raise Exception('error case')
        loss.backward()
        optimizer.step()
        model.apply(clipper)
        pbar.update()
pbar.close()
for name, param in model.named_parameters():
    print(f"{name}: {list(map(lambda x: round(float(x), 4),param))}")

train_set = dataset[dataset['i'] > 2]
epoch_len = len(train_set)
n_epoch = 1
print_len = max(epoch_len*n_epoch // 10, 1)
pbar = tqdm(desc="train", colour="red", total=epoch_len*n_epoch)

for k in range(n_epoch):
    for i, (_, row) in enumerate(shuffle(train_set, random_state=2022 + k).iterrows()):
        model.train()
        optimizer.zero_grad()
        output_t = [(model.zero, model.zero)]
        for input_t in row['tensor']:
            output_t.append(model(input_t, *output_t[-1]))
        loss = model.loss(output_t[-1][0], row['delta_t'],
                          {1: 0, 2: 1, 3: 1, 4: 1}[row['r']])
        if np.isnan(loss.data.item()):
            # Exception Case
            print(row, output_t)
            raise Exception('error case')
        loss.backward()
        for param in model.parameters():
            param.grad[:2] = torch.zeros(2)
        optimizer.step()
        model.apply(clipper)
        pbar.update()

        if (k * epoch_len + i) % print_len == 0:
            print(f"iteration: {k * epoch_len + i + 1}")
            for name, param in model.named_parameters():
                print(f"{name}: {list(map(lambda x: round(float(x), 4),param))}")
pbar.close()

w = list(map(lambda x: round(float(x), 4), dict(model.named_parameters())['w'].data))

print("\nTraining finished!")


100%|██████████| 56910/56910 [00:03<00:00, 15627.86it/s]


Tensorized!


pre—train: 100%|██████████| 5166/5166 [00:01<00:00, 4229.38it/s]


w: [1.1471, 1.4016, 5.0, -1.0, -1.0, 0.1, 1.5, -0.2, 0.8, 2.0, -0.2, 0.2, 1.0]


train:   0%|          | 35/51744 [00:00<02:33, 337.74it/s]

...

iteration: 41393
w: [1.1475, 1.401, 5.1195, -1.3066, -1.2153, 0.0485, 1.4812, -0.1294, 0.7698, 2.0114, -0.1864, 0.339, 1.0391]


train:  90%|█████████ | 46619/51744 [01:51<00:11, 450.40it/s]

iteration: 46567
w: [1.1475, 1.401, 5.1217, -1.3611, -1.2017, 0.0174, 1.4664, -0.1351, 0.7537, 1.9796, -0.218, 0.338, 0.9664]


train: 100%|██████████| 51744/51744 [02:02<00:00, 422.20it/s]

iteration: 51741
w: [1.1475, 1.401, 5.1481, -1.4223, -1.228, 0.0348, 1.4669, -0.1285, 0.754, 1.9673, -0.2305, 0.3203, 0.9453]

Training finished!


## 3 Result

Copy the optimal parameters for FSRS for you in the output of next code cell after running.

In [6]:
print(f"var w = {w};")

var w = [1.1475, 1.401, 5.1483, -1.4221, -1.2282, 0.035, 1.4668, -0.1286, 0.7539, 1.9671, -0.2307, 0.32, 0.9451];


You can see the memory states and intervals generated by FSRS as if you press the good in each review at the due date scheduled by FSRS.

In [7]:
requestRetention = 0.9  # recommended setting: 0.8 ~ 0.9


class Collection:
    def __init__(self):
        self.model = FSRS()

    def states(self, t_history, r_history):
        with torch.no_grad():
            line_tensor = lineToTensor(list(zip([t_history], [r_history]))[0])
            output_t = [(self.model.zero, self.model.zero)]
            for input_t in line_tensor:
                output_t.append(self.model(input_t, *output_t[-1]))
            return output_t[-1]


my_collection = Collection()
print("1:again, 2:hard, 3:good, 4:easy\n")
for first_rating in (1,2,3,4):
    print(f'first rating: {first_rating}')
    t_history = "0"
    d_history = "0"
    r_history = f"{first_rating}"  # the first rating of the new card
    # print("stability, difficulty, lapses")
    for i in range(10):
        states = my_collection.states(t_history, r_history)
        # print('{0:9.2f} {1:11.2f} {2:7.0f}'.format(
            # *list(map(lambda x: round(float(x), 4), states))))
        next_t = round(float(np.log(requestRetention)/np.log(0.9) * states[0]))
        difficulty = round(float(states[1]), 1)
        t_history += f',{int(next_t)}'
        d_history += f',{difficulty}'
        r_history += f",3"
    print(f"rating history: {r_history}")
    print(f"interval history: {t_history}")
    print(f"difficulty history: {d_history}")
    print('')


1:again, 2:hard, 3:good, 4:easy

first rating: 1
rating history: 1,3,3,3,3,3,3,3,3,3,3
interval history: 0,1,2,4,8,15,27,49,85,146,245
difficulty history: 0,8.0,7.9,7.8,7.7,7.6,7.5,7.4,7.4,7.3,7.2

first rating: 2
rating history: 2,3,3,3,3,3,3,3,3,3,3
interval history: 0,3,7,15,31,63,121,225,406,710,1210
difficulty history: 0,6.6,6.5,6.5,6.4,6.4,6.3,6.3,6.3,6.2,6.2

first rating: 3
rating history: 3,3,3,3,3,3,3,3,3,3,3
interval history: 0,4,11,27,62,134,276,542,1021,1853,3252
difficulty history: 0,5.1,5.1,5.1,5.1,5.1,5.1,5.1,5.1,5.1,5.1

first rating: 4
rating history: 4,3,3,3,3,3,3,3,3,3,3
interval history: 0,5,15,41,103,239,522,1076,2112,3966,7160
difficulty history: 0,3.7,3.8,3.8,3.9,3.9,4.0,4.0,4.0,4.1,4.1



You can change the `test_rating_sequence` to see the scheduling intervals in different ratings.

In [8]:
test_rating_sequence = "3,3,3,3,3,1,1,3,3,3,3,3"
requestRetention = 0.9  # recommended setting: 0.8 ~ 0.9
easyBonus = 1.3
hardInterval = 1.2

t_history = "0"
d_history = "0"
for i in range(len(test_rating_sequence.split(','))):
    rating = test_rating_sequence[2*i]
    last_t = int(t_history.split(',')[-1])
    r_history = test_rating_sequence[:2*i+1]
    states = my_collection.states(t_history, r_history)
    print(states)
    next_t = max(1,round(float(np.log(requestRetention)/np.log(0.9) * states[0])))
    if rating == '4':
        next_t = round(next_t * easyBonus)
    elif rating == '2':
        next_t = round(last_t * hardInterval)
    t_history += f',{int(next_t)}'
    difficulty = round(float(np.log(requestRetention)/np.log(0.9) * states[1]), 1)
    d_history += f',{difficulty}'
print(f"rating history: {test_rating_sequence}")
print(f"interval history: {t_history}")
print(f"difficulty history: {d_history}")

(tensor(3.9495), tensor(5.1483))
(tensor(10.6076), tensor(5.1483))
(tensor(26.7267), tensor(5.1483))
(tensor(61.8732), tensor(5.1483))
(tensor(134.3306), tensor(5.1483))
(tensor(6.5113), tensor(7.5187))
(tensor(2.3411), tensor(9.8062))
(tensor(3.1687), tensor(9.6432))
(tensor(4.4993), tensor(9.4858))
(tensor(6.3669), tensor(9.3340))
(tensor(9.2792), tensor(9.1875))
(tensor(13.7638), tensor(9.0462))
rating history: 3,3,3,3,3,1,1,3,3,3,3,3
interval history: 0,4,11,27,62,134,7,2,3,4,6,9,14
difficulty history: 0,5.1,5.1,5.1,5.1,5.1,7.5,9.8,9.6,9.5,9.3,9.2,9.0
